 # Helper functions for using the WOFOST model with Agrimetrics data

 © 2021 Agrimetrics

 ## Required libraries

In [1]:
import os
import math
import csv
import warnings
import numpy as np
import pandas as pd
from pcse.fileinput import (
    CSVWeatherDataProvider,
    CABOFileReader,
    YAMLAgroManagementReader,
)

import import_ipynb
from agri_graphql import fields, weather, centroid, soil



importing Jupyter notebook from agri_graphql.ipynb


 ## Data cache functions

In [2]:
def data_path():
    return os.path.join(os.path.abspath(""), "data")


def agrimetrics_data_path():
    return os.path.join(data_path(), "agrimetrics")


def wofost_data_path():
    return os.path.join(data_path, "wofost")


def short_id(field_id):
    short_id = field_id.replace("https://data.agrimetrics.co.uk/fields/", "").replace("agfd:", "")
    return short_id


def agri_wofost_weather_filepath(field_id, start_date, end_date):
    return os.path.join(agrimetrics_data_path(), short_id(field_id), f"weather_{start_date}_{end_date}.csv")


def agri_wofost_soil_filepath(field_id):
    return os.path.join(agrimetrics_data_path(), short_id(field_id), "soil.SOIL")


def agri_wofost_management_filepath(field_id, harvest_year):
    return os.path.join(agrimetrics_data_path(), short_id(field_id), f"{harvest_year}_management.yaml")



 ## Function to retrieve field identifiers

In [3]:
def field_ids(roi, dataset_id, sowncrop_dataset_id):
    return pd.DataFrame(fields(roi, dataset_id, sowncrop_dataset_id))



 ## Functions to transform the Agrimetrics weather data
 The weather data returned by the Agrimetrics GraphQL API query needs to be reformatted to be compatible with the WOFOST model.

In [4]:
def agri_wofost_weather_data(field_id, start_date, end_date):
    filepath = agri_wofost_weather_filepath(field_id, start_date, end_date)
    if not os.path.isfile(filepath):
        field_centroid = centroid(field_id)
        field_weather_data = pd.DataFrame(weather(field_id, start_date, end_date))
        field_weather_data = field_weather_data.pivot_table(
            values="value", index=["dateTime", "fieldId"], columns="dimension"
        ).reset_index()
        field_weather_data["dateTime"] = pd.to_datetime(field_weather_data["dateTime"])
        field_weather_data = field_weather_data.round(4)
        longitude, latitude = field_centroid["coordinates"]
        graphql_weather_as_wofost_data(filepath, longitude, latitude, field_weather_data)

    return CSVWeatherDataProvider(filepath, delimiter=",", force_reload=True)


def graphql_weather_as_wofost_data(filepath, longitude, latitude, records):
    os.makedirs(os.path.dirname(filepath), exist_ok=True)

    with open(filepath, "w", newline="") as f:
        weather_writer = csv.writer(f)
        weather_writer.writerows(
            [
                ["## Site Characteristics"],
                ["Country = 'UK'"],
                ["Station = 'Agrimetrics'"],
                ["Description = 'Weather observations for field from GraphQL'"],
                ["Source = 'Agrimetrics'"],
                ["Contact = 'developer@agrimetrics.co.uk'"],
                [
                    f"Longitude = {longitude}; Latitude = {latitude}; Elevation = 50; AngstromA = 0.18; AngstromB = 0.55; HasSnow= False; HasSunshine=False"
                ],
                ["## Daily weather observations (missing values are NaN)"],
                ["DAY", "IRRAD", "TMAX", "TMIN", "WIND", "RAIN", "VAP", "SNOWDEPTH"],
            ]
        )

        for _, row in records.iterrows():
            date = row["dateTime"]
            irrad = row["solarInsolationDaily"] * 3.6 * 1000  # kWh to kJ/day
            tmax = row["temperatureMaxDaily"]
            tmin = row["temperatureMinDaily"]
            rainfall = row["rainfallTotalDaily"]
            wind = row["windSpeedMeanDaily"]
            rh = row["relativeHumidityMeanDaily"]
            vap = calculate_vap(tmin, tmax, rh)
            snowdepth = math.nan

            out = [
                date.strftime("%Y%m%d"),
                irrad,
                tmax,
                tmin,
                wind,
                rainfall,
                vap,
                snowdepth,
            ]
            weather_writer.writerow(out)


def calculate_vap(tmin, tmax, rh):
    A = -1.044e4
    B = -11.29
    C = -2.7e-2
    D = 1.289e-5
    E = -2.478e-9
    F = 6.456
    T = ((tmax + tmin) / 2) * 1.8 + 491.67
    vap_sat = math.exp(A / T + B + C * T + D * T ** 2 + E * T ** 3 + F * math.log(T))
    vap = 6.89475729 * vap_sat * rh / 100  # PSI to kPa

    return vap



# Functions used to generate WOFOST soil files.
 Agrimetrics (top) soil texture and chemical properties data - retrieved from the GraphQL API - are used to calculate the required physical soil characteristics (soil water retention and hydraulic conductivity) listed in the file.
 Soil water characteristics estimated from soil texture and organic matter derived from Saxton and Rawls, 2006

In [5]:
def agri_wofost_soil_data(field_id, country):
    filepath = agri_wofost_soil_filepath(field_id)
    if not os.path.isfile(filepath):
        field_soil_data = soil(field_id)
        graphql_soil_as_wofost_data(field_id, field_soil_data, country)

    return CABOFileReader(filepath)


def graphql_soil_as_wofost_data(field_id, records, country):
    topsoil = records["topSoil"]
    sand_percentage = topsoil["texture"]["sandPercentage"] / 100
    clay_percentage = topsoil["texture"]["clayPercentage"] / 100
    mean_carbon_concs = {
        "FR": {
            "value": 261.0115167318557,
            "depth": "0-30 cm",
            "unit": "dg/kg",
            "conversion": 100,
        },
        "NL": {
            "value": 323.517958826106,
            "depth": "0-30 cm",
            "unit": "dg/kg",
            "conversion": 100,
        },
    }
    if country == "UK":
        carbon_concentration = topsoil["chemicalProperties"]["carbonConcentration"]["value"]
    else:
        carbon_data = mean_carbon_concs[country]
        carbon_concentration = carbon_data["value"] * carbon_data["conversion"]

    organic_matter_percentage = som(carbon_concentration)

    sat_soil = theta_s(sand_percentage, clay_percentage, organic_matter_percentage)
    fc_soil = theta_fc(sand_percentage, clay_percentage, organic_matter_percentage)
    pwp_soil = theta_pwp(sand_percentage, clay_percentage, organic_matter_percentage)

    filepath = agri_wofost_soil_filepath(field_id)
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, "w") as file:
        cabo_soil_file = f"""
** $Id: {field_id} $
**
** Wofost soil file from Agrimetrics data
**
** {field_id}

SOLNAM='{field_id}'

** physical soil characteristics

** soil water retention
SMW      =   {pwp_soil}  !  soil moisture content at wilting point [cm3/cm3]
SMFCF    =   {fc_soil}  !  soil moisture content at field capacity [cm3/cm3]
SM0      =   {sat_soil}  !  soil moisture content at saturation [cm3/cm3]
CRAIRC   =   0.090  ! critical soil air content for aeration [cm3/cm3]

** hydraulic conductivity
K0       =  {k_s(sat_soil, fc_soil)}  ! hydraulic conductivity of saturated soil [cm day-1]
KSUB     =  {k_s(sat_soil, fc_soil)}   ! maximum percolation rate subsoil [cm day-1]
SOPE     =  {k_s(sat_soil, fc_soil)}   ! maximum percolation rate root zone[cm day-1]
RDMSOL   =  150 ! maximum soil rootable depth [cm]
"""
        file.write(cabo_soil_file)


def theta_pwp(sand_frac, clay_frac, om_perc):
    """ Water content in root zone at saturation """
    sand_imp = -0.024 * sand_frac
    clay_imp = 0.487 * clay_frac
    om_imp = 0.006 * om_perc

    interactions = (0.005 * (sand_frac * om_perc)) - (0.013 * (clay_frac * om_perc)) + (0.068 * (sand_frac * clay_frac))

    theta_pwp_t = sand_imp + clay_imp + om_imp + interactions + 0.031

    return theta_pwp_t + (0.14 * theta_pwp_t - 0.02)


def som(carbon_concentration):
    """conversion of carbon concentration from milligrams per kilogram carbon to organic matter percentage"""
    carbon_concentration_percentage = 100 * carbon_concentration / (1000 * 1000)

    return carbon_concentration_percentage * 1.724


def theta_fc(sand_frac, clay_frac, om_perc):
    """ Water content in root zone at field capacity """
    sand_imp = -0.251 * sand_frac
    clay_imp = 0.195 * clay_frac
    om_imp = 0.011 * om_perc

    interactions = (0.006 * (sand_frac * om_perc)) - (0.027 * (clay_frac * om_perc)) + (0.452 * (sand_frac * clay_frac))

    theta_fc_t = sand_imp + clay_imp + om_imp + interactions + 0.299

    return theta_fc_t + ((1.283 * (theta_fc_t ** 2)) - (0.374 * theta_fc_t) - 0.015)


def theta_s_minus_fc(sand_frac, clay_frac, om_perc):
    """ Available water content in root zone """
    sand_imp = 0.278 * sand_frac
    clay_imp = 0.034 * clay_frac
    om_imp = 0.022 * om_perc

    interactions = (
        (-0.018 * (sand_frac * om_perc)) - (0.027 * (clay_frac * om_perc)) - (0.584 * (sand_frac * clay_frac))
    )

    first_soln = sand_imp + clay_imp + om_imp + interactions + 0.078

    return first_soln + (0.636 * first_soln - 0.107)


def theta_s(sand_frac, clay_frac, om_perc):
    """ Water content in root zone at saturation """
    fc = theta_fc(sand_frac, clay_frac, om_perc)
    aw = theta_s_minus_fc(sand_frac, clay_frac, om_perc)

    return fc + aw - (0.097 * sand_frac) + 0.043


def k_s(smc_sat, smc_fc):
    """ Saturated hydraulic conductivity """
    B = (np.log(1500) - np.log(33)) / (np.log(smc_fc) - np.log(smc_sat))
    lam = 1 / B
    ks = 1930 * (smc_sat - smc_fc) ** (3 - lam)
    return 24 * ks / 10  # mm/h to cm/day



 ## Functions to create the Agromanagement YAML file

In [6]:
def agri_wofost_management_data(field_id, harvest_year):
    filepath = agri_wofost_management_filepath(field_id, harvest_year)
    if not os.path.isfile(filepath):
        graphql_management_as_wofost_data(field_id, harvest_year)

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        return YAMLAgroManagementReader(filepath)


def graphql_management_as_wofost_data(field_id, harvest_year):
    filepath = agri_wofost_management_filepath(field_id, harvest_year)
    os.makedirs(os.path.dirname(filepath), exist_ok=True)

    harvest_year = int(harvest_year)
    yaml_str = f"""Version: 1.0
AgroManagement:
- {harvest_year - 1}-09-15:
    CropCalendar:
        crop_name: wheat
        variety_name: Winter_wheat_102
        crop_start_date: {harvest_year - 1}-10-15
        crop_start_type: sowing
        crop_end_date:
        crop_end_type: maturity
        max_duration: 350
    TimedEvents: null
    StateEvents: null"""

    with open(filepath, "w") as file:
        file.write(yaml_str)



# Referenes

 de Wit, Allard, Hendrik Boogaard, Davide Fumagalli, Sander Janssen, Rob Knapen, Daniel van Kraalingen, Iwan Supit, Raymond van der Wijngaart, and Kees van Diepen. "25 years of the WOFOST cropping systems model." Agricultural Systems 168 (2019): 154-167.

 K.E.Saxton & W.J.Rawls. "Soil Water Characteristic Estimates by Texture and Organic Matter for Hydrologic Solutions." Soil Science Society Of America Journal, Vol. 70.